In [1]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
import tqdm
import re
import ast
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count, freeze_support
import json

In [2]:
def get_items(items):
    for item in items.find_elements_by_class_name('item'):

        name = item.find_element_by_css_selector('span[itemprop="name"]').text
        url = item.find_element_by_css_selector('a[itemprop="url"]').get_attribute('href')
        
        try:
            price = int(item.find_element_by_class_name('price-value').get_attribute('data-price-value'))
        except:
            price = np.nan

        try:
            address = item.find_element_by_css_selector('span[itemprop="streetAddress"]').text
        except:
            address = ''

        try:
            rating = float(item.find_element_by_class_name('rating-value').text.split('/')[0])
        except:
            rating = np.nan

        try:
            rating_text = item.find_element_by_class_name('rating-text-description').text
        except:
            rating_text = ''

        try:
            reviews = int(re.findall(r'\d+', item.find_element_by_class_name('reviews').text)[0])
        except:
            reviews = np.nan

        try:
            stars = int(item.find_element_by_class_name('stars').get_attribute('class')[-1:])
        except:
            stars = np.nan 

        try:
            features = [el.text for el in item.find_elements_by_class_name('hotel_card__label_service')]
        except:
            features = '' 

        try:
            lat, lon = ast.literal_eval(item.find_element_by_css_selector('div[itemprop="address"]').get_attribute('data-hotel-coords'))
        except:
            lat = np.nan
            lon = np.nan
            
        try:
            meal = item.find_element_by_class_name('meal-type').text
        except:
            meal = ''

        local = ''
        dd_vars = {'аэропорта':np.nan, "ж/д вокзала":np.nan, 'пляжа':np.nan, "Байкала":np.nan, address.split(',')[0]:np.nan}
        for el in item.find_elements_by_class_name('distance'):
            obj = re.findall(r'(.*) \d+.*',el.text)[0].replace('До ', '').strip()
            try:
                dd_vars[obj] = re.findall(r'\d+.*',el.text)[0]
            except:
                dd_vars["Байкала"] = re.findall(r'\d+.*',el.text)[0]
        local = address.split(',')[0]
        
        hotels.append({
            'url':url,
            'name': name,
            'stars':stars,
            'price':price,
            'address': address,
            'lat':lat,
            'lon':lon,
            'features':features,
            'rating_text':rating_text,
            'rating':rating,
            'mark':reviews,
            'local':local,
            'dist_local':dd_vars[local],
            'dist_baikal': dd_vars['Байкала'],
            'dist_aeroport':dd_vars['аэропорта'],
            'dist_railway':dd_vars['ж/д вокзала'],
            'dist_beach':dd_vars['пляжа'],
            'meal_type':meal
        })

In [133]:
pages = []

In [237]:
items = driver.find_element_by_class_name('hotellist')
item = items.find_elements_by_class_name('item')[0]

In [3]:
hotels = []
driver = webdriver.Chrome()
for page in tqdm.tqdm_notebook(range(1, 26)):
    url = 'https://101hotels.com/russia/region/na_baikale/recreation_base?page='+str(page)
    driver.get(url)
    time.sleep(8)
    
    items = driver.find_element_by_class_name('hotellist')
    get_items(items)
    time.sleep(1)

<ipython-input-3-a7b2f81ac131>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm.tqdm_notebook(range(1, 26)):


In [5]:
df = pd.DataFrame(hotels)

In [8]:
df.drop_duplicates('url').shape[0]

525

In [18]:
df.name.str.lower().str.extract(r'([а-яa-z]+)').to_csv('data/title_hotels.csv', index = False)

In [12]:
df.loc[~df.dist_baikal.isna(), 'dist_baikal_meters'] = df.loc[~df.dist_baikal.isna()].dist_baikal.apply(lambda x: float(x.split(' ')[0])*1000 if 'км' in x else float(x.split(' ')[0]) )

In [14]:
df.to_csv('data/101hotels.csv', index = False)

## сбор доп инфы со страничек отелей

In [27]:
name_rating = ['Расположение',
  'Обслуживание',
  'Цена/качество',
  'Чистота',
  'Питание',
  'Качество сна']

In [121]:
def get_hotel(url, driver):
    driver.get(url)
    time.sleep(6)
    
    d = {'url':url}
    
    try:
        name_rating = [el.text for el in driver.find_element_by_class_name('summaries').find_elements_by_class_name('name')]
        score_rating = [float(el.text) for el in driver.find_element_by_class_name('summaries').find_elements_by_class_name('score')]
        
        for i in range(len(name_rating)):
            d[name_rating[i]] = score_rating[i]
    except:
        pass
    
    try:
        distances = [el.text.split(': ') for el in driver.find_element_by_class_name('hotel-distances').find_elements_by_class_name('tooltip')]
        
        for k, v in distances:
            d[k] = float(v.split(' ')[0]) * 1000 if 'км' in v else float(v.split(' ')[0])
    except:
        pass
            
    try:
        features = [el.get_attribute('title').replace('\xa0', ' ') for el in driver.find_element_by_class_name('services-list').find_elements_by_tag_name('li')]
        d['features'] = features
    except:
        pass
    
    try:
        capture = driver.find_element_by_class_name('hotel-rooms-number').text.split('. ')
        for i in capture:
            k,v = i.split(': ')
            d[k] = int(v)
    except:
        pass
    try:
        d['description'] = driver.find_element_by_css_selector('#description').text
        try:
            d['nearest_site'] = re.findall(r'Рядом с отелем есть:\n(.*)\n',driver.find_element_by_css_selector('#description').text)[0]
        except:
            pass
    except:
        pass
        
    result.append(d)
    time.sleep(2)

In [100]:
def start_pool(urls):
    driver = webdriver.Chrome()
    for url in tqdm.tqdm_notebook(urls, total = len(urls)):
        get_hotel(url, driver)
        time.sleep(5)

In [136]:
datalist = df.loc[~df.full_url.isin(df_f.url)].full_url
# result = []

if __name__== "__main__":
    freeze_support()
    pool = Pool()
    ITERATION_COUNT = cpu_count()-1 if len(datalist) >= cpu_count()-1 else len(datalist)

    count_per_iteration = len(datalist) / float(ITERATION_COUNT)

    for i in range(0, ITERATION_COUNT):
        list_start = int(count_per_iteration * i)
        list_end = int(count_per_iteration * (i+1))
        pool.apply_async(start_pool, [datalist[list_start:list_end]])

<ipython-input-100-a6526882ec76>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url in tqdm.tqdm_notebook(urls, total = len(urls)):


In [137]:
df_f = pd.DataFrame(result).drop_duplicates('url')

In [151]:
df.to_csv('data/101hotels.csv', index = False)

In [144]:
res = []
for i,r in df_f.iterrows():
    [r[df_f.drop(columns = 'features').columns]]

url         https://101hotels.com/main/cities/kurma/ekoote...
features    [Интернет, Бар / Ресторан, Автостоянка / Парко...
Name: 0, dtype: object
url         https://101hotels.com/main/cities/leestvyanka/...
features             [Интернет, Пляж, Автостоянка / Парковка]
Name: 1, dtype: object
url         https://101hotels.com/main/cities/leestvyanka/...
features    [Интернет, Бар / Ресторан, Прачечная или Химчи...
Name: 2, dtype: object
url         https://101hotels.com/main/cities/goryachinsk/...
features    [Интернет, Автостоянка / Парковка, Размещение ...
Name: 3, dtype: object
url         https://101hotels.com/main/cities/severobaykal...
features         [Интернет, Трансфер, Автостоянка / Парковка]
Name: 4, dtype: object
url         https://101hotels.com/main/cities/ust-barguzin...
features    [Бар / Ресторан, Автостоянка / Парковка, Бассе...
Name: 5, dtype: object
url         https://101hotels.com/main/cities/bolshie_koti...
features    [Прачечная или Химчистка, Автостоянка / Парк

Name: 63, dtype: object
url         https://101hotels.com/main/cities/huzhir/mini-...
features    [Интернет, Прачечная или Химчистка, Автостоянк...
Name: 64, dtype: object
url         https://101hotels.com/main/cities/suhaya/sukha...
features    [Сауна / SPA, Размещение домашних животных воз...
Name: 65, dtype: object
url         https://101hotels.com/main/cities/shida/monte_...
features    [Интернет, Бар / Ресторан, Автостоянка / Парко...
Name: 66, dtype: object
url         https://101hotels.com/main/cities/leestvyanka/...
features    [Интернет, Бар / Ресторан, Автостоянка / Парко...
Name: 67, dtype: object
url         https://101hotels.com/main/cities/huzhir/usadb...
features    [Интернет, Пляж, Автостоянка / Парковка, Транс...
Name: 68, dtype: object
url         https://101hotels.com/main/cities/huzhir/baika...
features    [Интернет, Бар / Ресторан, Автостоянка / Парко...
Name: 69, dtype: object
url         https://101hotels.com/main/cities/huzhir/mini-...
features    [Интернет, Пля

Name: 130, dtype: object
url         https://101hotels.com/main/cities/huzhir/guest...
features    [Пляж, Трансфер, Автостоянка / Парковка, Сауна...
Name: 131, dtype: object
url         https://101hotels.com/main/cities/kurma/baza_o...
features    [Бар / Ресторан, Трансфер, Автостоянка / Парко...
Name: 132, dtype: object
url         https://101hotels.com/main/cities/huzhir/goste...
features    [Автостоянка / Парковка, Размещение домашних ж...
Name: 133, dtype: object
url         https://101hotels.com/main/cities/bolshoe_golo...
features    [Прачечная или Химчистка, Трансфер, Автостоянк...
Name: 134, dtype: object
url         https://101hotels.com/main/cities/enheluk/gost...
features                   [Трансфер, Автостоянка / Парковка]
Name: 135, dtype: object
url         https://101hotels.com/main/cities/huzhir/olhon...
features                   [Трансфер, Автостоянка / Парковка]
Name: 136, dtype: object
url         https://101hotels.com/main/cities/kurma/kempin...
features    [Бар / 

Name: 210, dtype: object
url         https://101hotels.com/main/cities/baikalsk/tur...
features    [Пляж, Трансфер, Автостоянка / Парковка, Разме...
Name: 211, dtype: object
url         https://101hotels.com/main/cities/slyudyanka/d...
features    [Интернет, Прачечная или Химчистка, Автостоянк...
Name: 212, dtype: object
url         https://101hotels.com/main/cities/huzhir/goste...
features    [Автостоянка / Парковка, Размещение домашних ж...
Name: 213, dtype: object
url         https://101hotels.com/main/cities/goryachinsk/...
features    [Прачечная или Химчистка, Автостоянка / Парков...
Name: 214, dtype: object
url         https://101hotels.com/main/cities/sahyurta/baz...
features    [Интернет, Бар / Ресторан, Автостоянка / Парко...
Name: 215, dtype: object
url         https://101hotels.com/main/cities/sarma/baza_o...
features    [Бар / Ресторан, Пляж, Автостоянка / Парковка,...
Name: 216, dtype: object
url         https://101hotels.com/main/cities/sarma/komple...
features    [Интерн

Name: 287, dtype: object
url         https://101hotels.com/main/cities/sahyurta/gue...
features    [Интернет, Пляж, Автостоянка / Парковка, Транс...
Name: 288, dtype: object
url         https://101hotels.com/main/cities/slyudyanka/h...
features    [Интернет, Автостоянка / Парковка, Размещение ...
Name: 289, dtype: object
url         https://101hotels.com/main/cities/maksimiha/ts...
features    [Интернет, Трансфер, Автостоянка / Парковка, С...
Name: 290, dtype: object
url         https://101hotels.com/main/cities/sahyurta/gos...
features                       [Пляж, Автостоянка / Парковка]
Name: 291, dtype: object
url         https://101hotels.com/main/cities/sahyurta/soh...
features             [Интернет, Пляж, Автостоянка / Парковка]
Name: 292, dtype: object
url         https://101hotels.com/main/cities/nikola/gosti...
features    [Интернет, Бар / Ресторан, Автостоянка / Парко...
Name: 293, dtype: object
url         https://101hotels.com/main/cities/ust-barguzin...
features    [Автост

Name: 358, dtype: object
url         https://101hotels.com/main/cities/leestvyanka/...
features                                                  NaN
Name: 359, dtype: object
url         https://101hotels.com/main/cities/kurma/turbaz...
features    [Бар / Ресторан, Трансфер, Автостоянка / Парко...
Name: 360, dtype: object
url         https://101hotels.com/main/cities/huzhir/lakep...
features    [Интернет, Трансфер, Автостоянка / Парковка, Р...
Name: 361, dtype: object
url         https://101hotels.com/main/cities/huzhir/inn_o...
features                                                  NaN
Name: 362, dtype: object
url         https://101hotels.com/main/cities/ust-barguzin...
features    [Интернет, Трансфер, Автостоянка / Парковка, Р...
Name: 363, dtype: object
url         https://101hotels.com/main/cities/kurma/baza_o...
features    [Прачечная или Химчистка, Трансфер, Автостоянк...
Name: 364, dtype: object
url         https://101hotels.com/main/cities/posolskoe/ba...
features    [Сауна 

Name: 421, dtype: object
url         https://101hotels.com/main/cities/baikalsk/ski...
features    [Интернет, Бассейн, Автостоянка / Парковка, Пр...
Name: 422, dtype: object
url         https://101hotels.com/main/cities/baikalsk/gos...
features    [Интернет, Автостоянка / Парковка, Сауна / SPA...
Name: 423, dtype: object
url         https://101hotels.com/main/cities/sarma/uedine...
features    [Интернет, Пляж, Автостоянка / Парковка, Транс...
Name: 424, dtype: object
url         https://101hotels.com/main/cities/baikalsk/dom...
features    [Интернет, Бар / Ресторан, Автостоянка / Парко...
Name: 425, dtype: object
url         https://101hotels.com/main/cities/utulik/tayoz...
features    [Пляж, Трансфер, Автостоянка / Парковка, Фитне...
Name: 426, dtype: object
url         https://101hotels.com/main/cities/goryachinsk/...
features    [Интернет, Бар / Ресторан, Автостоянка / Парко...
Name: 427, dtype: object
url         https://101hotels.com/main/cities/gremyachinsk...
features    [Бар / 

Name: 495, dtype: object
url         https://101hotels.com/main/cities/huzhir/mini-...
features      [Трансфер, Автостоянка / Парковка, Сауна / SPA]
Name: 496, dtype: object
url         https://101hotels.com/main/cities/leestvyanka/...
features    [Интернет, Бар / Ресторан, Автостоянка / Парко...
Name: 497, dtype: object
url         https://101hotels.com/main/cities/huzhir/bayka...
features    [Интернет, Прачечная или Химчистка, Автостоянк...
Name: 498, dtype: object
url         https://101hotels.com/main/cities/baikalsk/fla...
features                                                  NaN
Name: 500, dtype: object
url         https://101hotels.com/main/cities/utulik/alpiy...
features                                                  NaN
Name: 501, dtype: object
url         https://101hotels.com/main/cities/huzhir/olkho...
features                   [Трансфер, Автостоянка / Парковка]
Name: 502, dtype: object
url         https://101hotels.com/main/cities/leestvyanka/...
features    [Интерн

In [147]:
sum(df_f.features.tolist(), [])

TypeError: can only concatenate list (not "float") to list

In [82]:
[el.text for el in driver.find_elements_by_tag_name('popupFacility')]

[]

In [91]:
re.findall(r'Рядом с отелем есть:\n(.*)\n',driver.find_element_by_css_selector('#description').text)[0]

'Музей Тальцы'

In [28]:
pd.DataFrame([{0:1, 1:2, 3:4}, {1:3, 3:2}])

,0,1,3
0,1.0,2,4
1,NaN,3,2


In [112]:
df.rename(columns = {'url':'full_url'}, inplace = True)

In [114]:
df['url'] = df.full_url.str.split('/').str[-1]

In [52]:
with open('search.json', 'r') as f:
    text = f.read()

In [54]:
response = json.loads(text)['response']

In [57]:
pd.json_normalize(response['hotels']).to_csv('data/101hotels_from_json.csv', index = False)

In [59]:
df2 = pd.json_normalize(response['hotels'])

In [33]:
df2 = pd.read_csv('data/101hotels_from_json.csv')
df = pd.read_csv('data/101hotels.csv')
df_f = pd.read_csv('data/hotel_metadata.csv')

C:\Users\yupes\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (57,112,113,114,117,118,121,122,123,126,127,152,153,155,157,159,161,163,165,167,255,256,260,262,264,266,268,270,272,274,276,278,280,282,284,286,288,290,297,298,299,300,305,306,308,310,312,314,316,318,320,354,356,358,360,362,364,371,373,375,377,379,381,383,385,387,389,391,393,395,397,399,401,403,404,409,433,435,437,439,441,451,452,454,456,458,460,462,464,466,468,470,489,490,492,494,496,498,500,502,504,506,508,512,514,516,518,520,522,524,526,529,531,533,535,537,539,541,543,545,547,549,551,553,555,557,559,561,563,565,567,569,571,578,579,581,583,585,587,589,599,600,602,604,606,608,610,612,614,622,624,626,628,630,632,634,636,638,640,642,644,653,654,659,660,662,664,666,668,675,677,681,682,684,686,688,690,692,694,696,698,700,702,704,706,708,710,712,714,716,718,720,722,724,726,728,733,734,736,738,742,743,744,746,748,750,752,754,756,758,760,764,765,767,769,778,779,781,783,785,787,789,791,799,

In [132]:
cols = []
for c in df2.columns:
    if 'categor' in c:
        cols.append(c)

In [36]:
df = pd.read_csv('data/101hotels_fetures.csv')

In [34]:
df = df.merge(df2[['url', 'category_ids']], on = 'url', how = 'left')

In [35]:
df = df.merge(df2[['url', 'number_reviews']], on = 'url', how = 'left')

In [38]:
df_f.loc[~df_f.features.isna(), 'features'] = df_f.loc[~df_f.features.isna()].features.apply(lambda x: ast.literal_eval(x))

In [13]:
df = df_f.rename(columns = {'url':'full_url'}).explode('features_item').drop(columns = 'features').merge(df, how = 'left', on = 'full_url')

In [43]:
df.loc[~df.features.isna(), 'features'] = df.loc[~df.features.isna()].features.apply(lambda x: ast.literal_eval(x))

In [17]:
df = df.explode('category_ids')

In [39]:
categories = ['Отели',
'Хостелы',
'Гостевые дома',
'Апартаменты',
'Пансионаты',
'Санатории',
'Базы отдыха',
'Коттеджи']
df['category'] = df['category_ids'].apply(lambda x: categories[x-1])

In [45]:
df.explode('features').to_csv('data/101hotels_features.csv', index = False)

тест

In [367]:
url = 'https://101hotels.com/russia/region/na_baikale/recreation_base?page=11'
driver.get(url)
time.sleep(5)

items = driver.find_element_by_class_name('hotellist')
for item in items.find_elements_by_class_name('item'):
    address = item.find_element_by_css_selector('span[itemprop="streetAddress"]').text
    local = ''
    dd_vars = {'аэропорта':np.nan, "ж/д вокзала":np.nan, 'пляжа':np.nan, "Байкала":np.nan, address.split(',')[0]:np.nan}
    for el in item.find_elements_by_class_name('distance'):
        obj = re.findall(r'(.*) \d+.*',el.text)[0].replace('До ', '').strip()
        try:
            dd_vars[obj] = re.findall(r'\d+.*',el.text)[0]
        except:
            dd_vars["Байкала"] = re.findall(r'\d+.*',el.text)[0]
    local = address.split(',')[0]

In [363]:
item.find_element_by_css_selector('a[itemprop="url"]').get_attribute('href')

'https://101hotels.com/main/cities/leestvyanka/majak.html'

In [312]:
address = item.find_element_by_css_selector('span[itemprop="streetAddress"]').text
local_distance = np.nan
aeroport_distance = np.nan 
rr_distance = np.nan
baikal_distance = np.nan
beach_distance = np.nan
local = ''
dd_vars = {'аэропорта':'aeroport_distance', "Ж/Д вокзала":'rr_distance', 'пляжа':'beach_distance', "Байкала":'baikal_distance', address.split(',')[0]:'local_distance'}
for el in item.find_elements_by_class_name('distance'):
    obj = re.findall(r'(.*) \d+.*',el.text)[0].replace('До ', '').strip()
#             print(el.text)
    try:
        print(obj.strip() == address.split(',')[0])
        v = dd_vars[obj]
        exec(v+"=re.findall(r'\d+.*','"+el.text+"')[0]")
#                 exec('print('+v+')')
    except:
        v = dd_vars["Байкала"]
        exec(v+"=re.findall(r'\d+.*','"+el.text+"')[0]")
local = obj

True


In [313]:
local_distance

'800 м'

In [287]:
re.findall(r'(.*) \d+.*',item.find_elements_by_class_name('distance')[0].text)[0]

['Сахюрта']

In [289]:
re.findall(r'(.*) \d+.*','До Шида 2.7 км')[0].replace('До ', '')

'Шида'

In [259]:
item.find_elements_by_class_name('distance')[0].text

'Сахюрта 800 м'

In [219]:
item.find('span', {'itemprop':'name'}).text

'Загородный комплекс Маяк'

In [89]:
', '.join([el.text for el in item.find_all('li', {'class':'hotel_card__label_service'})])

'Баня/сауна, Бассейн, Верховая езда, Детская площадка, Автостоянка'

In [96]:
ast.literal_eval(item.find('div', {'itemprop':'address'})['data-hotel-coords'])

[52.04336, 105.40833]

In [104]:
item.find('div', {'class':'stars'})['class'][1][-1:]

'3'

In [69]:
aeroport_distance, local

('59.2 км', 'Листвянка')

In [116]:
re.findall(r'До (.*) \d+.*',[el.text for el in item.find_all('span', {'class':'distance tooltip'})][-4])

['Шида']

In [150]:
[el.text for el in item.find_all('span', {'class':'distance tooltip'})]

['До Шида 2.7 км',
 'До аэропорта 147.7 км',
 'До Ж/Д вокзала 193.3 км',
 'До Залива Мухор 784 м']

# исправление

In [29]:
df = pd.read_csv('data/101hotels_features_v2.csv')

In [18]:
dff = df2.drop(columns = ['Расположение', 'Обслуживание','Цена/качество', 'Чистота', 'Питание', 'Качество сна']).merge(df[['full_url', 'Расположение', 'Обслуживание',
       'Цена/качество', 'Чистота', 'Питание', 'Качество сна']].drop_duplicates(), how = 'left', on = 'full_url')
# 

In [31]:
dff.loc[(dff['До Байкала']>10000) & (dff.local!='Патроны') & (dff.name != 'Мини-гостиница Твоя стихия')]['До Байкала_v2'] = dff.loc[(dff['До Байкала']>10000) & (dff.local!='Патроны') & (dff.name != 'Мини-гостиница Твоя стихия')]['До Байкала v2']/10

<ipython-input-31-52bcd980956a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff.loc[(dff['До Байкала']>10000) & (dff.local!='Патроны') & (dff.name != 'Мини-гостиница Твоя стихия')]['До Байкала_v2'] = dff.loc[(dff['До Байкала']>10000) & (dff.local!='Патроны') & (dff.name != 'Мини-гостиница Твоя стихия')]['До Байкала v2']/10


In [30]:
dff['До Байкала v2'] = dff['До Байкала'].copy()

In [119]:
dff.to_csv('data/101hotels_features_v2.csv', index = False)

In [33]:
dff[['До Байкала', 'До Байкала v2']]

,До Байкала,До Байкала v2
0,478.0,478.0
1,478.0,478.0
2,478.0,478.0
3,478.0,478.0
4,478.0,478.0
...,...,...
10015,300.0,300.0
10016,300.0,300.0
10017,300.0,300.0
10018,300.0,300.0


In [35]:
dff.loc[dff['До Байкала']> 100000][['До Байкала', "До Байкала v2", "dist_baikal"]]

,До Байкала,До Байкала v2,dist_baikal
3444,633900.0,633900.0,633.9 км
3445,633900.0,633900.0,633.9 км
3446,633900.0,633900.0,633.9 км
3447,633900.0,633900.0,633.9 км
3448,633900.0,633900.0,633.9 км
...,...,...,...
7136,4890300.0,4890300.0,4890.3 км
7137,4890300.0,4890300.0,4890.3 км
9325,297800.0,297800.0,297.8 км
9326,297800.0,297800.0,297.8 км


In [37]:
dff.loc[dff.url=='baykalskaya_zhemchuzhinka.html', 'До Байкала v2'] = 3700
dff.loc[dff.url=='zolotaya_rybka.html', 'До Байкала v2'] = 1400
dff.loc[dff.url=='gostinitsa_olimp_7.html', 'До Байкала v2'] = 3100
dff.loc[dff.url=='hotel_avrora_1.html', 'До Байкала v2'] = 600

In [38]:
dff.loc[dff.url=='svetlaya_polyana.html', 'До Байкала v2'] = 4000
dff.loc[dff.url=='baykalskiy_uyut_1.html', 'До Байкала v2'] = 3200
dff.loc[dff.url=='baykalsaya_nerpa.html', 'До Байкала v2'] = 1300
dff.loc[dff.url=='kulturno-ozdorovitelniy_tsentr_raduga.html', 'До Байкала v2'] = 1200

In [39]:
dff.loc[dff.url=='baykal_ersi_hotel.html', 'До Байкала v2'] = 5000
dff.loc[dff.url=='guesthouse_in_listvyanka.html', 'До Байкала v2'] = 5000
dff.loc[dff.url=='cottage_3.html', 'До Байкала v2'] = 4200

In [67]:
dff.groupby('local').max()[['До Байкала v2']]

<ipython-input-67-d7176d167579>:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  dff.groupby('local').max()[['До Байкала v2']]


,До Байкала v2
local,
Ангарские Хутора,5600.0
Байкал (порт),411.0
Байкальск,2500.0
Байкальский Прибой,NaN
Большие Коты,471.0
Большое Голоустное,2400.0
Бугульдейка,15000.0
Бурдугуз,NaN
Бухта Базарная,171.0


In [118]:
dff.loc[dff.url=='dom_na_baykale_3.html','lat'] = 53.100675
dff.loc[dff.url=='dom_na_baykale_3.html','lon'] = 106.834472

In [65]:
local = ['Шида', "Хужир", "Усть-Баргузин", "Сарма", "Курма", 'Сахюрта', "Максимиха"]

In [51]:
from IPython.display import display
import numpy as np

In [71]:
for l in local:
    dff.loc[(dff.local==l) & (~dff['До Байкала v2'].isna())& (dff['До Байкала v2'] >= 10000) , 'До Байкала v2'] = \
    np.median(dff.loc[(dff.local==l) & (~dff['До Байкала v2'].isna())& (dff['До Байкала v2'] < 10000) ]['До Байкала v2'].unique())
    print(l, dff.loc[(dff.local==l) & (~dff['До Байкала v2'].isna()) ]['До Байкала v2'].unique(),
         np.median(dff.loc[(dff.local==l) & (~dff['До Байкала v2'].isna())& (dff['До Байкала v2'] < 10000) ]['До Байкала v2'].unique()))

Шида [177.5 319.   36. ] 177.5
Хужир [ 826. 1300.  812. 1400.  445.  244. 1200. 1100.  416.  958.  487.] 826.0
Усть-Баргузин [4000. 3200.  125.] 3200.0
Сарма [4709.  118. 9300.] 4709.0
Курма [478.  505.5 533.  456.  710. ] 505.5
Сахюрта [352. 474. 350.  90.  57. 804. 967.  83. 267. 442. 632.] 352.0
Максимиха [ 782. 1300.   32.  364. 1200.] 782.0


In [97]:
(' '+dff.drop_duplicates('url').name+ ' ').str.lower().str.extractall(pat = r'([а-яa-zё]+)').reset_index()

In [101]:
df_title = (' '+dff.drop_duplicates('url').name+ ' ').str.lower().str.extractall(pat = r'([а-яa-zё]+)').reset_index()

In [99]:
import pymorphy2 as pm

In [100]:
morph = pm.MorphAnalyzer()

In [103]:
df_title['word'] = df_title[0].apply(lambda x: morph.parse(x)[0].normal_form)

In [104]:
df_title.to_csv('data/title_hotels.csv', index = False)